In [1]:
#organize training data, use type A and type B instrument
#already resize the pic to 300*300 with padding
#80% as train and 20% as test
import os
import tensorflow as tf
import numpy as np
from PIL import Image
train_folder = r'S:\\test\\instrument_resize_300'

def read_data(data_dir):
    image_list = []
    label_list = []
    for fname in os.listdir(data_dir):
        fpath = os.path.join(data_dir, fname)
        img_data = Image.open(fpath)
        if fname.startswith("A"):
            label_list.append(0)
        if fname.startswith("B"):
            label_list.append(1)
        data = np.array(img_data)
        image_list.append(data)
    return image_list,label_list

def get_train_test_data(image_list, label_list):
    temp = np.array([image_list,label_list])
    temp = temp.transpose()
    np.random.shuffle(temp)
    
    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    count = int(len(image_list) * 0.8)
    train_images = np.array(image_list[0:count])
    train_labels = np.array(label_list[0:count])
    test_images = np.array(image_list[count:])
    test_labels = np.array(label_list[count:])
    return train_images, train_labels, test_images, test_labels

In [3]:
#get training data and test data, divide 255.0 for normalization
from tensorflow import keras
image_list, label_list = read_data(train_folder)
train_images, train_labels, test_images, test_labels = get_train_test_data(image_list, label_list)

train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

train_images = train_images/255.0
test_images = test_images/255.0

In [4]:
#define model 
modelLet5 = keras.Sequential([
    keras.layers.Conv2D(input_shape=(300, 300, 3), filters=6, kernel_size=5, padding="same", activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=16, kernel_size=5, padding="same", activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation=tf.nn.relu),
    keras.layers.Dense(60, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])
modelLet5.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
#train model
modelLet5.fit(x=train_images, y=train_labels, epochs=5, batch_size=32, validation_split=0.1)

Train on 2086 samples, validate on 232 samples
Epoch 1/5
2086/2086 [==============================] - 161s 77ms/step - loss: 0.4083 - acc: 0.8126 - val_loss: 0.2379 - val_acc: 0.9224
Epoch 2/5
2086/2086 [==============================] - 158s 76ms/step - loss: 0.0921 - acc: 0.9679 - val_loss: 0.1814 - val_acc: 0.9181
Epoch 3/5
2086/2086 [==============================] - 158s 76ms/step - loss: 0.0309 - acc: 0.9899 - val_loss: 0.1835 - val_acc: 0.9483
Epoch 4/5
2086/2086 [==============================] - 163s 78ms/step - loss: 0.0479 - acc: 0.9818 - val_loss: 0.1688 - val_acc: 0.9440
Epoch 5/5
2086/2086 [==============================] - 162s 78ms/step - loss: 0.0224 - acc: 0.9928 - val_loss: 0.2066 - val_acc: 0.9440


In [6]:
#test
score = modelLet5.evaluate(test_images, test_labels)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

580/580 [==============================] - 21s 35ms/step
Test Loss: 0.17179937947718105
Test accuracy: 0.9517241379310345
